# Outil : filtrer les entrées du dictionnaire enrichi de Buchanan

Amélie Leboeuf

### Importer les librairies

In [312]:
from IPython.display import display, clear_output, HTML, Markdown, Audio # import necessaire à la création des widgets
import ipywidgets as widgets # import necessaire à la création des widgets
import pandas as pd
from bs4 import BeautifulSoup # Isoler des partie et naviguer dans le code HTML
import requests # Requête HTML
#from IPython import display # Ecouter un mp3

### Importer le dictionnaire enrichi de Buchanan

In [320]:
cheminDuFichierCSV = "./../02_OUTPUT/DicoTradBuch_ALL.csv" 
dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")
#dfDicoBuch.sample(n=5)

### Scrapping

In [314]:
def mp3(word):
    urlTarget = "https://www.linguee.fr/anglais-francais/traduction/"+ word +".html" # Site web
    response = requests.get(urlTarget) # Requête
    soup = BeautifulSoup(response.text,'html.parser') 
    souplettesBody = soup.find('body') # Rechercher tout le contenu des balise body
    finded = souplettesBody.find('a',{"class":"audio"})
    value = finded["id"][5:len(finded["id"])]
    return value

### Création des widget

In [315]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="love",
       description='motVedette',
       layout={'width': '300px'})

In [316]:
sliderNbSyll = widgets.IntRangeSlider(
    value=[0, 7],
    min=0,
    max=7,
    step=1,
    description='nbSyll')

In [317]:
sliderTarget = widgets.BoundedIntText( 
    value=2,
    min=1,
    max=20,
    step=1,
    description='Taille')

In [318]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):

    with out1:
        clear_output()
        
        df = dfDicoBuch[dfDicoBuch["motVedette"].str.contains(textTarget.value)]
        if sliderNbSyll.value[0] == sliderNbSyll.value[1]:
            df = df[df["nbSyll"] == sliderNbSyll.value[0]]
        else :
            df = df[df["nbSyll"] >= sliderNbSyll.value[0]]
            df = df[df["nbSyll"] <= sliderNbSyll.value[1]]
        df = df.head(sliderTarget.value)
        display(df)

    with out2:
        clear_output()
        
        for word in df["motVedette"]:
            value = mp3(word)
            #value = "/3e/3e63afd0565f7fa98d084139112a04d5-400" # test
            lienUS = "https://www.linguee.fr/mp3/EN_US" + value +".mp3" # Lien US
            lienUK = "https://www.linguee.fr/mp3/EN_UK" + value +".mp3" # Lien UK
            
            prep = "US : " + word
            display(prep, Audio(url=lienUS))
            prep = "UK : " + word
            display(prep, Audio(url=lienUK))
        
        
button.on_click(on_button_clicked)

### Afficher l'outil

In [319]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget, sliderNbSyll, sliderTarget, button])

# les widgets d'output
vbOuts = widgets.VBox([out1,out2])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts])
hbAllWidgets